In [ ]:
import random
import numpy as np
import pandas as pd
from ortools.sat.python import cp_model
from math import ceil

class Problem:

    quotas = None
    staff_leave_allowance = None
    preference_matrix = None

    def __init__(self, num_staff: int, num_days: int, quotaLimits: tuple = None, quotaArray: list = None, leaveAllowanceLimits: tuple = None, leaveAllowanceArray: list = None, preferencePercentage:int = None, preferenceMatrix: list = None):
        self.num_staff = num_staff
        self.num_days = num_days            

        if quotaLimits:
            self.quotas = [((random.randrange(quotaLimits[0], quotaLimits[1]) / 100)) for i in range(self.num_days)] 

        elif quotaArray:
            if not len(quotaArray) == self.num_days:
                raise ValueError("Quota array must have the same length as the number of days in the problem")
            
            self.quotas = quotaArray

        if leaveAllowanceLimits:
            self.staff_leave_allowance = [random.randint(leaveAllowanceLimits[0], leaveAllowanceLimits[1]) for i in range(self.num_staff)]

        elif leaveAllowanceArray:
            if not len(leaveAllowanceArray) == self.num_staff:
                raise ValueError("Leave allowance array must have the same length as the number of staff in the problem")
            
            self.staff_leave_allowance = leaveAllowanceArray

        if not preferenceMatrix:
            self.generate_preference_matrix(preferencePercentage)
        else:    
            self.preference_matrix = np.array(preferenceMatrix).reshape(self.num_staff, self.num_days)

        if not self.quotas:
            raise ValueError("Quotas must be defined")
        
        if not self.staff_leave_allowance:
            raise ValueError("Staff leave allowance must be defined")
        
        if self.preference_matrix.any() == None:
            raise ValueError("Preference matrix must be defined")
        
    def __str__(self):
        return f"{self.num_staff} staff, {self.num_days} days,\nquotas:\n{self.quotas},\nstaff leave allowance:\n{self.staff_leave_allowance}\npreference matrix:\n{self.preference_matrix}"
        

    def generate_preference_matrix(self, preferencePercentage):
        num_ones = int(self.num_staff * self.num_days * (preferencePercentage / 100))
        num_zeros = (self.num_staff * self.num_days) - num_ones
        array = np.array([1] * num_ones + [0] * num_zeros)
        np.random.shuffle(array)
        self.preference_matrix = array.reshape(self.num_staff,self.num_days)

    def to_csv(self, filename):
        ## ADD OTHER METRICS TO FILE
        df = pd.DataFrame(self.preference_matrix)
        df.to_csv(filename, index=False, header=False)

    # def from_csv(self, filename):
        # df = pd.read_csv(filename, header=None)

    def createModel(self):
        model = cp_model.CpModel()


        # Matrix for leave granted
        # if employee e has day d off, then L[e,d] = 1
        l = {}
        for e in range(self.num_staff):
            for d in range(self.num_days):
                l[(e, d)] = model.new_bool_var(f"L_{e}_d{d}")

        # Staff leave allowance not exceeded
        for e in range(self.num_staff):
            model.Add(sum(l[e, d] for d in range(self.num_days)) <= self.staff_leave_allowance[e])
        
        # daily leave quota not exceeded
        # calculate number of staff * staff_limit for each day
        for d in range(self.num_days):
            model.Add(sum(l[e, d] for e in range(self.num_staff)) <= ceil(self.num_staff * self.staff_limit[d]))

        pass

In [ ]:
import solara
import solara.lab

num_staff = solara.reactive(10)
num_days = solara.reactive(5)
leave_entitlement_range = solara.reactive((1,10))
staff_threshold_range = solara.reactive((25,50))
percentage_preference = solara.reactive(40)

problem = solara.reactive(None)

def new_problem():

    global num_staff, num_days, leave_entitlement_range, staff_threshold_range, percentage_preference, problem

    print(num_staff, num_days, leave_entitlement_range, staff_threshold_range, percentage_preference)
    
    problem.value = Problem(num_staff=num_staff.value, num_days=num_days.value, leaveAllowanceLimits=leave_entitlement_range.value, quotaLimits=staff_threshold_range.value, preferencePercentage=percentage_preference.value)

    print(problem)

@solara.component
def Page():
    with solara.Column():
        solara.Title("Dissertation Project")
        with solara.Sidebar():
            with solara.Card("Controls", margin=0, elevation=0):
                with solara.Column():
                    with solara.Row():
                        solara.Button("Use Sample dataset", color="primary", text=True, outlined=True)
                        
                    solara.InputInt(label="Number of staff", value=num_staff)
                    solara.InputInt(label="Number of days", value=num_days)

                    solara.SliderRangeInt("Leave Entitlement", value=leave_entitlement_range, min=0, max=365)
                    solara.Markdown(f"Min Value: **{leave_entitlement_range.value[0]}**, Max Value: **{leave_entitlement_range.value[1]}**")

                    solara.SliderRangeInt("Daily Absence Threshold", value=staff_threshold_range, min=0, max=100, )
                    solara.Markdown(f"Min Value: **{staff_threshold_range.value[0]}**, Max Value: **{staff_threshold_range.value[1]}**")

                    solara.InputInt(label="Percentage of leave requested", value=percentage_preference)
                    
                    solara.Button("Generate data", color="primary", text=True, outlined=True, on_click=new_problem)

        with solara.Card("Modelling Data"):
            with solara.Columns([1]):
                if problem.value:
                    with solara.Column():
                        with solara.Row():
                            solara.Info(f"Employees: {problem.value.num_staff}", color="#000000", icon=False, dense=True)
                            solara.Info(f"Days: {problem.value.num_days}", color="#000000", icon=False, dense=True)

                        solara.Info(f"Leave allowances: {problem.value.staff_leave_allowance}", color="#000000", icon=False, dense=False)
                        solara.Info(f"Daily Staff Leave Quotas: {problem.value.quotas}", color="#000000", icon=False, dense=False)

                        with solara.Card("Preference Matrix"):
                            solara.DataFrame(pd.DataFrame(problem.value.preference_matrix, columns=[f'Day {d+1}' for d in range(problem.value.num_days)], index=[f'Employee {x+1}' for x in range(problem.value.num_staff)]), items_per_page=10)

                            with pd.option_context('display.max_colwidth', None,
                       'display.max_columns', None,
                       'display.max_rows', None):

                                display(pd.DataFrame(problem.value.preference_matrix, columns=[f'Day {d+1}' for d in range(problem.value.num_days)], index=[f'Employee {x+1}' for x in range(problem.value.num_staff)]))
                else:
                    solara.Markdown("No data to display")


                    